In [120]:
import pandas as pd
import numpy as np
import os 
import geopandas as gpd
import plotly.express as px

In [121]:
mf = gpd.read_file('./Marine_forest/Marine_forest.shp')
count = len(mf[(mf['면적'] == mf['구역']) & (mf['면적'] == mf['규모'])])
print(count)
mf.drop(columns=['구역','규모'],inplace=True)
col_names = {'시도명': 'province','시군구명':'city','시군구코드':'city_code','위도':'latitude','경도':'longitude','마을어장':'town','면적':'area'}
mf.rename(columns=col_names, inplace=True)
#기본 칼럼인 위도 경도는 오류가 있었음
mf['latitude'] = mf['geometry'].y
mf['longitude'] = mf['geometry'].x
# mf = pd.DataFrame(mf)
# mf.to_csv('./Marine_forest/mf.csv', index=False)
mf

219


,province,city,city_code,town,latitude,longitude,시설년도,시설유형,품종,area,ORIG_FID,geometry
0,충청남도,서산시,21,가로림만1단지,36.918893,126.335300,2013,어초,감태,3.640241e+05,0,POINT (126.33530 36.91889)
1,충청남도,서산시,21,가로림만2단지,36.910919,126.353197,2013,어초,감태,2.338867e+05,1,POINT (126.35320 36.91092)
2,충청남도,서산시,21,가로림만3단지,36.896696,126.348674,2013,어초,감태,4.164415e+05,2,POINT (126.34867 36.89670)
3,강원도,고성군,82,가진리,38.368750,128.518750,2018,어초,미역,1.514584e+06,3,POINT (128.51875 38.36875)
4,강원도,삼척시,23,갈남리,37.281873,129.329315,2012,어초,미역,3.635963e+05,4,POINT (129.32931 37.28187)
...,...,...,...,...,...,...,...,...,...,...,...,...
214,강원도,고성군,82,고성군 청간리,38.268750,128.568750,2022,어초,미역,1.516629e+06,214,POINT (128.56875 38.26875)
215,강원도,속초시,21,속초시 대포동,38.168750,128.618750,2022,어초,감태,1.518669e+06,215,POINT (128.61875 38.16875)
216,경상북도,포항시,11,포항시 하정2리,35.968426,129.557133,2022,어초,미역,1.308860e+06,216,POINT (129.55713 35.96843)
217,경상북도,포항시,11,포항시 계원1리,35.865283,129.534375,2022,어초,감태,1.025338e+06,217,POINT (129.53438 35.86528)


In [122]:
# mf_2013 = pd.DataFrame(mf.loc[mf['시설년도']<= 2013])
# mf_2014 = pd.DataFrame(mf.loc[mf['시설년도']<= 2014])
# mf_2015 = pd.DataFrame(mf.loc[mf['시설년도']<= 2015])
# mf_2016 = pd.DataFrame(mf.loc[mf['시설년도']<= 2016])
# mf_2017 = pd.DataFrame(mf.loc[mf['시설년도']<= 2017])
# mf_2018 = pd.DataFrame(mf.loc[mf['시설년도']<= 2018])
# mf_2019 = pd.DataFrame(mf.loc[mf['시설년도']<= 2019])
# mf_2020 = pd.DataFrame(mf.loc[mf['시설년도']<= 2020])

전체에 적용할 기본적인 전처리

In [123]:
# Ocean_data 폴더 내의 모든 CSV 파일 목록을 가져옵니다.
folder_path = 'Ocean_data'
all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and f.startswith('19') or f.startswith('20')]
# 제거하려는 열의 리스트

# 각 파일을 순차적으로 읽어 데이터 프레임 리스트에 추가합니다.
dfs = []
for filename in all_files:
    cols_to_drop = ['dep_flag', 'Valid_time','project_name',]
    file_path = os.path.join(folder_path, filename)
    df = pd.read_csv(file_path)
    ##Longitude:위도 , Latitude: 경도
    colnames = {'yyyy-mm-dd hh:mm:ss': 'date_time', 
                'Longitude [degrees_east]': 'longitude', 
                'Latitude [degrees_north]': 'latitude',
                '수심(m)': 'depth(m)',
                '수심(text)': 'depth(text)',
                '유효시간': 'Valid_time',
                
                }
    df.rename(columns=colnames, inplace=True)

    data = df.copy()
    #데이터 품질값이 1~3인 값만 남기고 나머지는 NaN으로 대체합니다.
    qc_cols = [(idx, col) for idx, col in enumerate(data.columns) if col.endswith('_qc')]
    for idx, col in qc_cols:
        obs_df = data.iloc[:, [idx - 1, idx]].copy()
        obs_df.iloc[~obs_df.iloc[:, 1].isin([1, 2,3]), 0] = np.nan
        col_value = data.columns[idx - 1]
        data[col_value] = obs_df.iloc[:, 0]
    #qc칼럼 제거
    qc_cols = [col for col in data.columns if col.endswith('_qc')]
    data.drop(columns=qc_cols, inplace=True)

    data['date_time'] = pd.to_datetime(data['date_time'],format='mixed')
    # data['year'] = data['date_time'].dt.year
    # data['month'] = data['date_time'].dt.month
    # data['quarter'] = data['date_time'].dt.quarter
    # data['year_quarter'] = data['year'].astype(str) + data['quarter'].astype(str)
    # 데이터프레임에 해당 열이 있는지 확인하고 제거합니다.
    cols_to_drop = [col for col in cols_to_drop if col in data.columns]
    data = data.drop(columns=cols_to_drop)
    data.to_csv(f'./data/{filename[:4]}.csv',index = False)

첫번째 전처리

In [124]:
def first_preprocess(origin_path, loc, cols, name, mode = True):
    # 원본 경로에서 모든 csv 파일을 가져옴
    folder_path = origin_path
    all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and f.startswith('19') or f.startswith('20')]
   
    for filename in all_files:
        # 제거할 열 목록
        cols_to_drop = cols
        # 파일 경로 생성
        file_path = os.path.join(folder_path, filename)
        # csv 파일을 읽어서 데이터프레임으로 저장
        df = pd.read_csv(file_path,encoding='utf-8')
        # mode가 True인 경우, depth(m)이 0인 행만 선택
        if mode :
            df = df.loc[(df['depth(m)'] == 0)]
        # mode가 False인 경우, depth(m)이 0이 아닌 행만 선택
        else:
            df = df.loc[(df['depth(m)'] != 0)]
        # 선택된 열 중에서 cols_to_drop에 있는 열 제거
        cols_to_drop = [col for col in cols_to_drop if col in df.columns]
        df = df.drop(columns=cols_to_drop)
        # 결과 데이터프레임을 csv 파일로 저장
        df.to_csv(f'{loc}/{filename[:4]}_{name}_df.csv',index= False)

표층데이터

In [125]:
first_preprocess(origin_path = 'data',loc = './data/surface',cols=['depth(m)','depth(text)','전체수심(m)','bot.depth[m]'], name = "surface", mode = True)

저층 데이터

In [126]:
first_preprocess(origin_path = 'data',loc = './data/low',cols=['depth(text)','전체수심(m)','bot.depth[m]'], name = "low", mode = False)

바다숲과 가까이 있는 해양데이터포인터 찾기

In [134]:
from scipy.spatial import KDTree
def radius_by_marineforest(folder_path):
    mf = gpd.read_file('./Marine_forest/Marine_forest.shp')
    count = len(mf[(mf['면적'] == mf['구역']) & (mf['면적'] == mf['규모'])])
    mf.drop(columns=['구역','규모'],inplace=True)
    col_names = {'시도명': 'province','시군구명':'city','시군구코드':'city_code','위도':'latitude','경도':'longitude','마을어장':'town','면적':'area'}
    mf.rename(columns=col_names, inplace=True)
    #기본 칼럼인 위도 경도는 오류가 있었음
    mf['latitude'] = mf['geometry'].y
    mf['longitude'] = mf['geometry'].x
    # 특정 반경 (예: 0.01도)
    radius = 0.01

    output_folder = f'./{folder_path}/{radius}'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    folder_path = folder_path
    all_dfs = []  # 모든 데이터프레임을 저장할 리스트
    for year in ['2012','2013','2014','2015','2016', '2017', '2018', '2019', '2020']:
        # 해당 연도의 모든 CSV 파일 목록을 가져옵니다.
        all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and int(f[:4]) <= int(year)]
        # 해당 연도 이전의 mf 데이터만 필터링합니다.
        filtered_mf = mf[mf['시설년도'] <= int(year)]
        print(len(filtered_mf),end=" ")
        # 필터링된 mf 데이터를 KDTree에 로드합니다.
        tree = KDTree(filtered_mf[['latitude', 'longitude']].values)
        print(all_files)
        for filename in all_files:
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path, encoding='utf-8')

            # mf의 각 좌표에 대해 주어진 반경 내의 모든 df의 좌표의 인덱스를 찾습니다.
            indices_list = tree.query_ball_point(df[['latitude', 'longitude']].values, r=radius)

            # 각 near_df 행에 대한 겹치는 mf 좌표의 수를 저장하는 리스트를 초기화합니다.
            overlap_counts = []
                # 모든 인덱스를 하나의 배열로 펼칩니다.
            all_indices = [idx for sublist in indices_list for idx in sublist]

            # 중복 제거
            unique_indices = np.unique(all_indices)

            # 선택된 인덱스를 사용하여 near_df 생성
            near_df = df.iloc[unique_indices].reset_index(drop=True)

            # 각 near_df 행에 대해 겹치는 mf 좌표의 수를 계산합니다.
            for idx in unique_indices:
                count = sum([1 for sublist in indices_list if idx in sublist])
                overlap_counts.append(count)
                
            # 겹치는 mf 좌표의 수를 near_df에 새로운 칼럼으로 추가합니다.
            near_df['overlap_count'] = overlap_counts

            all_dfs.append(near_df)

        # 모든 데이터프레임을 하나로 합칩니다.
        combined_df = pd.concat(all_dfs, ignore_index=True)
        combined_df.to_csv(os.path.join(output_folder, f'{filename[:4]}.csv'),index=False)



col = ['투명도, 총인, 총질소, 부유물질 농도, 염분, 화학적산소요구량, 클로로필 a','용존산소량']

In [135]:
mf = gpd.read_file('./Marine_forest/Marine_forest.shp')
count = len(mf[(mf['면적'] == mf['구역']) & (mf['면적'] == mf['규모'])])
mf.drop(columns=['구역','규모'],inplace=True)
col_names = {'시도명': 'province','시군구명':'city','시군구코드':'city_code','위도':'latitude','경도':'longitude','마을어장':'town','면적':'area'}
mf.rename(columns=col_names, inplace=True)
#기본 칼럼인 위도 경도는 오류가 있었음
mf['latitude'] = mf['geometry'].y
mf['longitude'] = mf['geometry'].x
year = 2009
filtered_mf = mf[mf['시설년도'] <= int(year)]
filtered_mf

,province,city,city_code,town,latitude,longitude,시설년도,시설유형,품종,area,ORIG_FID,geometry
38,경상남도,거제시,31,다대리,34.734299,128.639300,2009,어초,미역,2.468894e+05,38,POINT (128.63930 34.73430)
39,경상남도,거제시,31,다포리,34.720410,128.633059,2009,어초,감태,1.916612e+05,39,POINT (128.63306 34.72041)
70,제주특별자,서귀포시,13,사계리,33.212303,126.308813,2009,어초,감태,2.014052e+06,70,POINT (126.30881 33.21230)
71,강원도,강릉시,15,사근진,37.822134,128.892569,2009,어초,감태,4.730294e+05,71,POINT (128.89257 37.82213)
95,경상북도,포항시,11,신창2리,35.885820,129.532696,2009,어초,감태,3.509255e+05,95,POINT (129.53270 35.88582)
153,충청남도,태안군,82,파도리,36.722149,126.141421,2009,어초,감태,1.234693e+03,153,POINT (126.14142 36.72215)


In [136]:
radius_by_marineforest('./data/surface')
radius_by_marineforest('./data/low')

37 ['2007_surface_df.csv', '2008_surface_df.csv', '2009_surface_df.csv', '2010_surface_df.csv', '2011_surface_df.csv', '2012_surface_df.csv']
48 ['2007_surface_df.csv', '2008_surface_df.csv', '2009_surface_df.csv', '2010_surface_df.csv', '2011_surface_df.csv', '2012_surface_df.csv', '2013_surface_df.csv']
67 ['2007_surface_df.csv', '2008_surface_df.csv', '2009_surface_df.csv', '2010_surface_df.csv', '2011_surface_df.csv', '2012_surface_df.csv', '2013_surface_df.csv', '2014_surface_df.csv']
87 ['2007_surface_df.csv', '2008_surface_df.csv', '2009_surface_df.csv', '2010_surface_df.csv', '2011_surface_df.csv', '2012_surface_df.csv', '2013_surface_df.csv', '2014_surface_df.csv', '2015_surface_df.csv']
112 ['2007_surface_df.csv', '2008_surface_df.csv', '2009_surface_df.csv', '2010_surface_df.csv', '2011_surface_df.csv', '2012_surface_df.csv', '2013_surface_df.csv', '2014_surface_df.csv', '2015_surface_df.csv', '2016_surface_df.csv']
129 ['2007_surface_df.csv', '2008_surface_df.csv', '2009_su

하나의 큰 파일 만들기

In [130]:
# Ocean_data 폴더 내의 모든 CSV 파일 목록을 가져옵니다.
folder_path = 'data'
all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and f.startswith('19') or f.startswith('20')]

# 각 파일을 순차적으로 읽어 데이터 프레임 리스트에 추가합니다.
dfs = []
for filename in all_files:
    file_path = os.path.join(folder_path, filename)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 모든 데이터 프레임을 하나로 합칩니다.
merged_data = pd.concat(dfs, ignore_index=True)
merged_data.to_csv('./data/merged_data.csv',index = False)

In [131]:
# missing_values_per_column = merged_data.isnull().sum()
# print(missing_values_per_column)


In [132]:
# surface_df = merged_data.loc[merged_data['수심(text)'] == '표층']
# surface_df.drop(columns=['수심(m)','수심(text)'],inplace=True)
# surface_df.isnull().sum()
# surface_df.to_csv('./data/surface_df.csv',index= False)


In [133]:
# columns_to_plot = ['총질소[μg/L]', '총인[μg/L]', '용존무기질소[μg/L]','화학적산소요구량[mg/L]','암모니아성 질소[μg/L]']

# fig = px.scatter(low_df, x='year_quarter', y='총질소[μg/L]', title='Values over Year Quarter')
# fig.show()